# 회귀 기반 추천 시스템

In [1]:
!pip install catboost

In [2]:
import pandas as pd

In [3]:
travel_df = pd.read_csv('./data/travel_1.csv')
print(travel_df)
travel_df.head()

      GENDER  AGE_GRP  TRAVEL_STYL_1  TRAVEL_STYL_2  TRAVEL_STYL_3  \
0          남     30.0            1.0            4.0            2.0   
1          남     20.0            4.0            1.0            5.0   
2          여     50.0            4.0            1.0            2.0   
3          남     30.0            1.0            1.0            1.0   
4          여     20.0            5.0            3.0            3.0   
...      ...      ...            ...            ...            ...   
34567      여     20.0            1.0            2.0            2.0   
34568      남     30.0            3.0            3.0            3.0   
34569      남     40.0            2.0            2.0            5.0   
34570      여     30.0            1.0            1.0            1.0   
34571      남     30.0            2.0            2.0            2.0   

       TRAVEL_STYL_4  TRAVEL_STYL_5  TRAVEL_STYL_6  TRAVEL_STYL_7  \
0                2.0            6.0            2.0            2.0   
1                1.0 

,GENDER,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,VISIT_AREA_NM,MVMN_NM,DGSTFN
0,남,30.0,1.0,4.0,2.0,2.0,6.0,2.0,2.0,7.0,3.0,3.0,미스틱3도,자가용,5.0
1,남,20.0,4.0,1.0,5.0,1.0,1.0,4.0,1.0,6.0,3.0,1.0,에스제이렌트카,대중교통 등,4.0
2,여,50.0,4.0,1.0,2.0,4.0,3.0,3.0,2.0,3.0,1.0,3.0,법환식당,대중교통 등,5.0
3,남,30.0,1.0,1.0,1.0,5.0,6.0,3.0,5.0,7.0,7.0,3.0,에코랜드호텔,자가용,5.0
4,여,20.0,5.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0,1.0,2.0,윤스타피자앤파스타,자가용,4.0


In [4]:
travel_df[['AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM']] = travel_df[['AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM']].astype(int)

travel_df.head()

,GENDER,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,VISIT_AREA_NM,MVMN_NM,DGSTFN
0,남,30,1,4,2,2,6,2,2,7,3,3,미스틱3도,자가용,5.0
1,남,20,4,1,5,1,1,4,1,6,3,1,에스제이렌트카,대중교통 등,4.0
2,여,50,4,1,2,4,3,3,2,3,1,3,법환식당,대중교통 등,5.0
3,남,30,1,1,1,5,6,3,5,7,7,3,에코랜드호텔,자가용,5.0
4,여,20,5,3,3,3,3,3,3,5,1,2,윤스타피자앤파스타,자가용,4.0


In [5]:
from sklearn.model_selection import train_test_split

x = travel_df.drop('DGSTFN', axis=1)
y = travel_df['DGSTFN']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [6]:
from catboost import Pool

cat_features = [
    'GENDER', 'AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM', 'VISIT_AREA_NM', 'MVMN_NM'
]

X_train_pool = Pool(x_train, y_train, cat_features=cat_features)
x_test_pool = Pool(x_test, y_test, cat_features=cat_features)

In [7]:
from catboost import CatBoostRegressor

cb_reg = CatBoostRegressor(
    n_estimators=100,   # 반복횟수(내부 예측기 개수)
    depth=5,             # 개별 트리의 최대 깊이
    learning_rate=0.03,
    loss_function='RMSE',
    eval_metric='RMSE'
)

cb_reg.fit(X_train_pool, eval_set=x_test_pool, verbose=10)

0:	learn: 0.8348252	test: 0.8360693	best: 0.8360693 (0)	total: 183ms	remaining: 18.1s
10:	learn: 0.8309097	test: 0.8313066	best: 0.8313066 (10)	total: 398ms	remaining: 3.22s
20:	learn: 0.8284089	test: 0.8281688	best: 0.8281688 (20)	total: 602ms	remaining: 2.27s
30:	learn: 0.8266281	test: 0.8259954	best: 0.8259954 (30)	total: 801ms	remaining: 1.78s
40:	learn: 0.8253390	test: 0.8242814	best: 0.8242814 (40)	total: 1.01s	remaining: 1.45s
50:	learn: 0.8241862	test: 0.8230994	best: 0.8230994 (50)	total: 1.22s	remaining: 1.17s
60:	learn: 0.8232098	test: 0.8221181	best: 0.8221181 (60)	total: 1.41s	remaining: 903ms
70:	learn: 0.8224472	test: 0.8213216	best: 0.8213216 (70)	total: 1.6s	remaining: 653ms
80:	learn: 0.8216428	test: 0.8205121	best: 0.8205121 (80)	total: 1.81s	remaining: 424ms
90:	learn: 0.8210018	test: 0.8198809	best: 0.8198809 (90)	total: 1.98s	remaining: 195ms
99:	learn: 0.8203473	test: 0.8193664	best: 0.8193664 (99)	total: 2.14s	remaining: 0us

bestTest = 0.8193664476
bestIteratio

In [8]:
col_importance = pd.DataFrame({
    'column': x_train.columns,
    'importance': cb_reg.feature_importances_
})

col_importance

,column,importance
0,GENDER,0.771593
1,AGE_GRP,14.510488
2,TRAVEL_STYL_1,5.528368
3,TRAVEL_STYL_2,8.485650
4,TRAVEL_STYL_3,6.954300
5,TRAVEL_STYL_4,3.464870
6,TRAVEL_STYL_5,4.786226
7,TRAVEL_STYL_6,7.467811
8,TRAVEL_STYL_7,1.247661
9,TRAVEL_STYL_8,11.272213


##### 추천 시스템 구축

1. 방문지 목록을 생성
2. 사용자 특성 입력
3. 가상 만족도 예측
4. 만족도가 높은 순으로 추천

In [10]:
visit_areas = travel_df['VISIT_AREA_NM'].unique()
visit_areas.shape
visit_areas[:10]

array(['미스틱3도', '에스제이렌트카', '법환식당', '에코랜드호텔', '윤스타피자앤파스타', '서정리역 1호선',
       '동문식당', '해녀식당', '알맞은시간', '애월온기'], dtype=object)

In [12]:
user_input = ['여', 60, 4, 4, 4, 4, 4, 4, 4, 4, 1, 2, '방문지', '자가용']
pred_results = []

for area in visit_areas:
    user_input[-2] =area
    dgstfn_pred = cb_reg.predict(user_input)
    pred_results.append(dgstfn_pred)

pred_results[:10]

[4.306107212369755,
 4.134672247211243,
 4.256663752108525,
 4.2466825925724025,
 4.088213814198101,
 4.0794092856928925,
 4.088213814198101,
 4.141207260897324,
 4.256663752108525,
 4.200830663195209]

In [13]:
result_df = pd.DataFrame({
    'VISIT_AREA_NM': visit_areas,
    'DGSTFN_PRED': pred_results
})

result_df.sort_values(by='DGSTFN_PRED', ascending=False).head(10)

,VISIT_AREA_NM,DGSTFN_PRED
193,여울목게스트하우스,4.308569
3450,한라산국립공원 영실탐방로,4.308569
1037,올레길 21코스,4.308569
676,파라다이스시티,4.308569
216,제주신라호텔,4.308569
60,산지해장국,4.308569
3288,귤꽃다락,4.308569
1303,영종씨사이드 레일바이크,4.308569
78,제주1번가더테라스오피스텔,4.308569
3936,한라산국립공원 관음사탐방로,4.308569
